## Import Libraries

In [1]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np

In [2]:
pwd= os.getcwd()

In [3]:
df1 = pd.read_csv(pwd + "/_202305041824.csv")
df_countrytiers = pd.read_excel(pwd + "/Tier1_List.xlsx")
df_managerslist = pd.read_excel(pwd + "/ManagersList.xlsx")
df_campmanagerslist = pd.read_excel(pwd + "/CampManagersList.xlsx")

In [4]:
index = df1[ (df1['COST'] == 0) & (df1['REVENUE'] ==0) ].index
df1.drop(index , inplace=True)

In [5]:
import re
words = ['suite', 'workspace','ads','advertis','video']
regex = '|'.join(map(re.escape, words))
# 'health|therapist|sales|private'
# keywords- lms, 3pl, edi,headless, knowledge/content, endpoint, server,project, health
# new keywords - soc 2, it , headless , discovery, billing, staffing, inventory
# dormant - laboratory, Supply Chain, corporate, therapist, event management, identity, power bi, medical staffing, ,forensic, ehr, enterprise
#lawyer,marketing,order, file, therap, analytics, document, cyber, workflow, ai, consulting

#removed 23aug - email,private
#removed 21Sept- mental,document
#added -ads, workable
#'product','task', 'ats' , gcp

# option 1: get first match
df1['category_first'] = (df1['Campaign_Query']
 .str.extract(f'(?i)({regex})', expand=False)
 .fillna('Other')
 )

# option 2: get all matches
df1['category_all'] = (df1['Campaign_Query']
 .str.extractall(f'(?i)({regex})')
 [0].groupby(level=0).agg(lambda x: ','.join(set(x)))
 .reindex(df1.index, fill_value='Other')
 )

### Merge two strings into ONE - suite and workspace

In [6]:
#df1['category_first'] = df1['category_first'].replace(['suite', 'workspace','Workspace','ads','advertis','video'], 'G Suite')

In [7]:
#df1.pivot_table(['COST','REVENUE'], index='category_first', aggfunc='sum', margins=True)

,COST,REVENUE
category_first,,
G Suite,33811.43,44092.43
Other,15857.19,18996.10
Suite,86.56,110.80
All,49755.18,63199.33


#### Merge CountryList

In [4]:
df_merge_1 = pd.merge(df1, df_countrytiers, on='Country', how='left')

In [5]:
df_merge_1['Tier'] =  df_merge_1['Tier'].fillna('NO')

In [6]:
df_merge_1.pivot_table(['COST','REVENUE'], index='Tier', aggfunc='sum', margins=True)

,COST,REVENUE
Tier,,
NO,336.76,411.79
Tier 1,206.52,226.41
All,543.28,638.20


In [7]:
df_merge_1.to_excel("RAW_FAA_Overview_May_04.xlsx", index=False)